### Import Libraries

In [1]:
import numpy as np
import pandas as pd
import keras
from keras import initializers
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from keras.layers import *
from keras.models import *
from keras import backend as K
import os
import shutil
import pathlib

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
from sklearn.model_selection import train_test_split

!pip install tensorflow-addons
import tensorflow_addons as tfa
tf.get_logger().setLevel('ERROR')



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Loading data

In [2]:
df = pd.read_csv('data/labelled_pol.csv')

In [3]:
df.dropna(inplace = True)

### Balance class weights

In [4]:
pos = df[df['Label'] == 1]
neut = df[df['Label'] == 0]
l = df['Label'].value_counts()[0]
df = pd.concat([pos.head(l), neut])

In [5]:
def compute_class_weights(labels):

    # Count number of postive and negative bags.
    negative_count = len(np.where(labels == 0)[0])
    positive_count = len(np.where(labels == 1)[0])
    total_count = negative_count + positive_count

    # Build class weight dictionary.
    return {
        0: (1 / negative_count) * (total_count / 2),
        1: (1 / positive_count) * (total_count / 2),
    }


### Split data

In [6]:
# Use a utility from sklearn to split and shuffle your dataset.
train_df, test_df = train_test_split(df, test_size=0.2)
train_df, val_df = train_test_split(train_df, test_size=0.2)

In [7]:
### Define Metrics

In [8]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision 

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [9]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = [tf.metrics.BinaryAccuracy(), f1_m, precision_m, recall_m]
callback = tf.keras.callbacks.EarlyStopping(monitor = 'val_f1_m', patience = 3)

### Encode Data

In [10]:
bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8'  #@param ["bert_en_uncased_L-12_H-768_A-12", "bert_en_cased_L-12_H-768_A-12", "bert_multi_cased_L-12_H-768_A-12", "small_bert/bert_en_uncased_L-2_H-128_A-2", "small_bert/bert_en_uncased_L-2_H-256_A-4", "small_bert/bert_en_uncased_L-2_H-512_A-8", "small_bert/bert_en_uncased_L-2_H-768_A-12", "small_bert/bert_en_uncased_L-4_H-128_A-2", "small_bert/bert_en_uncased_L-4_H-256_A-4", "small_bert/bert_en_uncased_L-4_H-512_A-8", "small_bert/bert_en_uncased_L-4_H-768_A-12", "small_bert/bert_en_uncased_L-6_H-128_A-2", "small_bert/bert_en_uncased_L-6_H-256_A-4", "small_bert/bert_en_uncased_L-6_H-512_A-8", "small_bert/bert_en_uncased_L-6_H-768_A-12", "small_bert/bert_en_uncased_L-8_H-128_A-2", "small_bert/bert_en_uncased_L-8_H-256_A-4", "small_bert/bert_en_uncased_L-8_H-512_A-8", "small_bert/bert_en_uncased_L-8_H-768_A-12", "small_bert/bert_en_uncased_L-10_H-128_A-2", "small_bert/bert_en_uncased_L-10_H-256_A-4", "small_bert/bert_en_uncased_L-10_H-512_A-8", "small_bert/bert_en_uncased_L-10_H-768_A-12", "small_bert/bert_en_uncased_L-12_H-128_A-2", "small_bert/bert_en_uncased_L-12_H-256_A-4", "small_bert/bert_en_uncased_L-12_H-512_A-8", "small_bert/bert_en_uncased_L-12_H-768_A-12", "albert_en_base", "electra_small", "electra_base", "experts_pubmed", "experts_wiki_books", "talking-heads_base"]
map_name_to_handle = {'small_bert/bert_en_uncased_L-4_H-512_A-8': 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'}
map_model_to_preprocess = {'small_bert/bert_en_uncased_L-4_H-512_A-8': 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'}
tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


In [11]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

### Encoding Data

In [12]:
x_train = bert_preprocess_model(train_df['Lemmatized_Text'])['input_word_ids']
x_val = bert_preprocess_model(val_df['Lemmatized_Text'])['input_word_ids']
x_test = bert_preprocess_model(test_df['Lemmatized_Text'])['input_word_ids']

In [13]:
# high = max(x_train[0])
# for i in x_train:
#     if max(i) > high:
#         high = max(i)
# for i in x_test:
#     if max(i) > high:
#         high = max(i)

### Padding Encodings

In [14]:
n_unique_words = 29603 # cut texts after this number of words
maxlen = 50
batch_size = 128

In [15]:
x = sequence.pad_sequences(x_train, maxlen=maxlen)
x_val = sequence.pad_sequences(x_val, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

y = np.array(train_df['Label'])
y_val = np.array(val_df['Label'])
y_test = np.array(test_df['Label'])


### Defining model

In [16]:
def build_LSTM():
    model = Sequential()
    model.add(Embedding(n_unique_words, 128, input_length=maxlen))
    model.add(Bidirectional(LSTM(64)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=metrics) 
    return model

In [17]:
class attention(Layer):
    def __init__(self, return_sequences=True):
        self.return_sequences = return_sequences

        super(attention,self).__init__()

    def build(self, input_shape):
        self.W=self.add_weight(name="att_weight", shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1), initializer="zeros")
        super(attention,self).build(input_shape)


    def call(self, x):
        e = K.tanh(K.dot(x,self.W)+self.b)
        a = K.softmax(e, axis=1)
        output = x*a
        if self.return_sequences:
            return output
        return K.sum(output, axis=1)

In [18]:
def build_Att_Bi_LSTM():
    model = Sequential()
    model.add(Embedding(n_unique_words, 128, input_length=maxlen))
    model.add(Bidirectional(LSTM(64,return_sequences=True)))
    model.add(attention(return_sequences=False))
    model.add(Dense(1, activation='sigmoid'))
    opt = tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name='Adam',
)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=metrics) 
    return model

### Training

In [19]:
checkpoint_path = "weights/polarity.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

### Training Bi-LSTM Model

In [20]:
from sklearn.model_selection import KFold
acc = []
cv = KFold(n_splits=5, shuffle=True, random_state=42)
k_train = np.concatenate((x, x_val), axis=0)
k_labels = np.concatenate((y, y_val), axis=0)
for train_idx, test_idx in cv.split(k_train,k_labels):
    
        x_train, y_train  = k_train[train_idx],k_labels[train_idx]
        x_val, y_val= k_train[test_idx], k_labels[test_idx]
        
        acc_ = []

        model = build_LSTM()

        history = model.fit(x_train, y_train , validation_data=[x_val, y_val],
                               epochs=12,
                               class_weight=compute_class_weights(y_train),
                               callbacks=[cp_callback])

        acc_.append(history.history['val_binary_accuracy'][-1])
        
        acc.append(acc_)
    
cv_acc = np.mean(np.array(acc), axis = 0)

Epoch 1/12
11/12 [==========================>...] - ETA: 0s - loss: 0.6923 - binary_accuracy: 0.5170 - f1_m: 0.5834 - precision_m: 0.5510 - recall_m: 0.7644
Epoch 1: saving model to weights\polarity.ckpt
12/12 [==============================] - 6s 177ms/step - loss: 0.6932 - binary_accuracy: 0.5104 - f1_m: 0.5765 - precision_m: 0.5392 - recall_m: 0.7542 - val_loss: 0.6892 - val_binary_accuracy: 0.5464 - val_f1_m: 0.4610 - val_precision_m: 0.3854 - val_recall_m: 0.5760
Epoch 2/12
11/12 [==========================>...] - ETA: 0s - loss: 0.6814 - binary_accuracy: 0.5568 - f1_m: 0.6000 - precision_m: 0.5480 - recall_m: 0.6919
Epoch 2: saving model to weights\polarity.ckpt
12/12 [==============================] - 1s 76ms/step - loss: 0.6820 - binary_accuracy: 0.5495 - f1_m: 0.5929 - precision_m: 0.5398 - recall_m: 0.6842 - val_loss: 0.6859 - val_binary_accuracy: 0.5670 - val_f1_m: 0.4587 - val_precision_m: 0.3987 - val_recall_m: 0.5404
Epoch 3/12
11/12 [==========================>...] - ETA

13/13 [==============================] - 1s 75ms/step - loss: 0.5255 - binary_accuracy: 0.7169 - f1_m: 0.7894 - precision_m: 0.6611 - recall_m: 1.0000 - val_loss: 1.1182 - val_binary_accuracy: 0.5000 - val_f1_m: 0.5340 - val_precision_m: 0.4711 - val_recall_m: 0.6183
Epoch 7/12
13/13 [==============================] - ETA: 0s - loss: 0.5212 - binary_accuracy: 0.7065 - f1_m: 0.7710 - precision_m: 0.6489 - recall_m: 0.9790
Epoch 7: saving model to weights\polarity.ckpt
13/13 [==============================] - 1s 69ms/step - loss: 0.5212 - binary_accuracy: 0.7065 - f1_m: 0.7710 - precision_m: 0.6489 - recall_m: 0.9790 - val_loss: 0.7512 - val_binary_accuracy: 0.5104 - val_f1_m: 0.6022 - val_precision_m: 0.4844 - val_recall_m: 0.7964
Epoch 8/12
13/13 [==============================] - ETA: 0s - loss: 0.4821 - binary_accuracy: 0.7169 - f1_m: 0.7137 - precision_m: 0.5846 - recall_m: 0.9231
Epoch 8: saving model to weights\polarity.ckpt
13/13 [==============================] - 1s 73ms/step - 

Epoch 12/12
12/13 [==========================>...] - ETA: 0s - loss: 0.4629 - binary_accuracy: 0.6927 - f1_m: 0.7504 - precision_m: 0.6053 - recall_m: 1.0000
Epoch 12: saving model to weights\polarity.ckpt
13/13 [==============================] - 1s 73ms/step - loss: 0.4618 - binary_accuracy: 0.6935 - f1_m: 0.7696 - precision_m: 0.6357 - recall_m: 1.0000 - val_loss: 0.8471 - val_binary_accuracy: 0.6250 - val_f1_m: 0.6709 - val_precision_m: 0.6116 - val_recall_m: 0.7597
Epoch 1/12
12/13 [==========================>...] - ETA: 0s - loss: 0.6939 - binary_accuracy: 0.5146 - f1_m: 0.6211 - precision_m: 0.5061 - recall_m: 0.9125
Epoch 1: saving model to weights\polarity.ckpt
13/13 [==============================] - 6s 162ms/step - loss: 0.6938 - binary_accuracy: 0.5156 - f1_m: 0.6502 - precision_m: 0.5441 - recall_m: 0.9193 - val_loss: 0.6947 - val_binary_accuracy: 0.4792 - val_f1_m: 0.6314 - val_precision_m: 0.4869 - val_recall_m: 0.9167
Epoch 2/12
12/13 [==========================>...] - E

13/13 [==============================] - ETA: 0s - loss: 0.5671 - binary_accuracy: 0.6831 - f1_m: 0.7665 - precision_m: 0.6388 - recall_m: 0.9828
Epoch 5: saving model to weights\polarity.ckpt
13/13 [==============================] - 1s 77ms/step - loss: 0.5671 - binary_accuracy: 0.6831 - f1_m: 0.7665 - precision_m: 0.6388 - recall_m: 0.9828 - val_loss: 0.6876 - val_binary_accuracy: 0.5938 - val_f1_m: 0.6662 - val_precision_m: 0.5272 - val_recall_m: 0.9063
Epoch 6/12
13/13 [==============================] - ETA: 0s - loss: 0.5039 - binary_accuracy: 0.7039 - f1_m: 0.7779 - precision_m: 0.6496 - recall_m: 1.0000
Epoch 6: saving model to weights\polarity.ckpt
13/13 [==============================] - 1s 81ms/step - loss: 0.5039 - binary_accuracy: 0.7039 - f1_m: 0.7779 - precision_m: 0.6496 - recall_m: 1.0000 - val_loss: 0.6944 - val_binary_accuracy: 0.5625 - val_f1_m: 0.6250 - val_precision_m: 0.5085 - val_recall_m: 0.8159
Epoch 7/12
12/13 [==========================>...] - ETA: 0s - loss:

In [21]:
model.evaluate(x_test, y_test)

4/4 [==============================] - 2s 44ms/step - loss: 0.9710 - binary_accuracy: 0.5868 - f1_m: 0.6684 - precision_m: 0.5982 - recall_m: 0.7602


[0.9710254669189453,
 0.586776852607727,
 0.6683982610702515,
 0.5982456207275391,
 0.7601503729820251]

### Training Attention based BiLSTM model

In [22]:
from sklearn.model_selection import KFold
acc = []
cv = KFold(n_splits=5, shuffle=True, random_state=42)
k_train = np.concatenate((x, x_val), axis=0)
k_labels = np.concatenate((y, y_val), axis=0)
for train_idx, test_idx in cv.split(k_train,k_labels):
    
        x_train, y_train  = k_train[train_idx],k_labels[train_idx]
        x_val, y_val= k_train[test_idx], k_labels[test_idx]
        
        acc_ = []

        model = build_Att_Bi_LSTM()

        history = model.fit(x_train, y_train , validation_data=[x_val, y_val],
                               epochs=12,
                               class_weight=compute_class_weights(y_train),
                               callbacks=[cp_callback])

        acc_.append(history.history['val_binary_accuracy'][-1])
        
        acc.append(acc_)
    
cv_acc = np.mean(np.array(acc), axis = 0)

Epoch 1/12
12/12 [==============================] - ETA: 0s - loss: 0.6902 - binary_accuracy: 0.5545 - f1_m: 0.4961 - precision_m: 0.4048 - recall_m: 0.6514
Epoch 1: saving model to weights\polarity.ckpt
12/12 [==============================] - 6s 192ms/step - loss: 0.6902 - binary_accuracy: 0.5545 - f1_m: 0.4961 - precision_m: 0.4048 - recall_m: 0.6514 - val_loss: 0.7098 - val_binary_accuracy: 0.5104 - val_f1_m: 0.5718 - val_precision_m: 0.4405 - val_recall_m: 0.8337
Epoch 2/12
11/12 [==========================>...] - ETA: 0s - loss: 0.6851 - binary_accuracy: 0.5767 - f1_m: 0.6478 - precision_m: 0.5516 - recall_m: 0.8040
Epoch 2: saving model to weights\polarity.ckpt
12/12 [==============================] - 1s 95ms/step - loss: 0.6824 - binary_accuracy: 0.5885 - f1_m: 0.6579 - precision_m: 0.5600 - recall_m: 0.8151 - val_loss: 0.6922 - val_binary_accuracy: 0.5104 - val_f1_m: 0.5481 - val_precision_m: 0.4324 - val_recall_m: 0.7550
Epoch 3/12
11/12 [==========================>...] - ETA

12/12 [==============================] - ETA: 0s - loss: 0.4851 - binary_accuracy: 0.7240 - f1_m: 0.7794 - precision_m: 0.6414 - recall_m: 1.0000
Epoch 6: saving model to weights\polarity.ckpt
12/12 [==============================] - 1s 100ms/step - loss: 0.4851 - binary_accuracy: 0.7240 - f1_m: 0.7794 - precision_m: 0.6414 - recall_m: 1.0000 - val_loss: 1.0847 - val_binary_accuracy: 0.4688 - val_f1_m: 0.5228 - val_precision_m: 0.4518 - val_recall_m: 0.6321
Epoch 7/12
12/12 [==============================] - ETA: 0s - loss: 0.4769 - binary_accuracy: 0.7266 - f1_m: 0.7721 - precision_m: 0.6349 - recall_m: 1.0000
Epoch 7: saving model to weights\polarity.ckpt
12/12 [==============================] - 1s 88ms/step - loss: 0.4769 - binary_accuracy: 0.7266 - f1_m: 0.7721 - precision_m: 0.6349 - recall_m: 1.0000 - val_loss: 1.1674 - val_binary_accuracy: 0.4896 - val_f1_m: 0.5510 - val_precision_m: 0.4694 - val_recall_m: 0.6815
Epoch 8/12
12/12 [==============================] - ETA: 0s - loss

12/12 [==============================] - 1s 77ms/step - loss: 0.4565 - binary_accuracy: 0.7005 - f1_m: 0.7489 - precision_m: 0.6056 - recall_m: 1.0000 - val_loss: 0.9490 - val_binary_accuracy: 0.5938 - val_f1_m: 0.7063 - val_precision_m: 0.5943 - val_recall_m: 0.9052
Epoch 12/12
11/12 [==========================>...] - ETA: 0s - loss: 0.4489 - binary_accuracy: 0.6960 - f1_m: 0.7444 - precision_m: 0.6046 - recall_m: 1.0000
Epoch 12: saving model to weights\polarity.ckpt
12/12 [==============================] - 1s 89ms/step - loss: 0.4501 - binary_accuracy: 0.6979 - f1_m: 0.7442 - precision_m: 0.6035 - recall_m: 1.0000 - val_loss: 0.8627 - val_binary_accuracy: 0.5625 - val_f1_m: 0.6852 - val_precision_m: 0.5833 - val_recall_m: 0.8763
Epoch 1/12
11/12 [==========================>...] - ETA: 0s - loss: 0.6912 - binary_accuracy: 0.5223 - f1_m: 0.4156 - precision_m: 0.3463 - recall_m: 0.5278
Epoch 1: saving model to weights\polarity.ckpt
12/12 [==============================] - 6s 228ms/step

Epoch 5/12
11/12 [==========================>...] - ETA: 0s - loss: 0.5743 - binary_accuracy: 0.6591 - f1_m: 0.7029 - precision_m: 0.6312 - recall_m: 0.8839
Epoch 5: saving model to weights\polarity.ckpt
12/12 [==============================] - 1s 86ms/step - loss: 0.5710 - binary_accuracy: 0.6641 - f1_m: 0.7110 - precision_m: 0.6341 - recall_m: 0.8936 - val_loss: 0.5789 - val_binary_accuracy: 0.6875 - val_f1_m: 0.7369 - val_precision_m: 0.6033 - val_recall_m: 0.9556
Epoch 6/12
12/12 [==============================] - ETA: 0s - loss: 0.5470 - binary_accuracy: 0.6745 - f1_m: 0.7447 - precision_m: 0.5969 - recall_m: 1.0000
Epoch 6: saving model to weights\polarity.ckpt
12/12 [==============================] - 1s 79ms/step - loss: 0.5470 - binary_accuracy: 0.6745 - f1_m: 0.7447 - precision_m: 0.5969 - recall_m: 1.0000 - val_loss: 0.5626 - val_binary_accuracy: 0.6562 - val_f1_m: 0.7176 - val_precision_m: 0.5765 - val_recall_m: 0.9556
Epoch 7/12
11/12 [==========================>...] - ETA:

In [23]:
model.evaluate(x_test, y_test)

4/4 [==============================] - 1s 21ms/step - loss: 0.8569 - binary_accuracy: 0.5785 - f1_m: 0.6957 - precision_m: 0.5764 - recall_m: 0.8806


[0.8569455742835999,
 0.5785123705863953,
 0.6957402229309082,
 0.5763888955116272,
 0.8806390762329102]